In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import psycopg2
from psycopg2 import sql

In [2]:
# AWS RDS Connection
PGEND_POINT = 'gameanalysis.cjhwxnjx5697.us-west-1.rds.amazonaws.com' # End_point
PGDATABASE_NAME = 'gameanalysis' # Database Name example: youtube_test_db
PGUSER_NAME = 'postgres' # UserName

In [3]:
# Connect to AWS RDS
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ PGEND_POINT +" port="+ "5432" +" user=" + PGUSER_NAME \
                  +" password="+ PGPASSWORD
    
    conn = psycopg2.connect(conn_string)
    print("Connected!")

    # Create a cursor object
    cursor = conn.cursor()
    
    return conn, cursor

In [4]:
#Check Connection
conn, cursor = connect()

Connected!


CHAMPIONS MASTERY

In [5]:
# Select champion mastery table
query_championMastery = sql.SQL("""
SELECT * FROM "championMastery";
""")

In [6]:
cur = conn.cursor()
cur.execute(query_championMastery)

In [7]:
df_championMastery = pd.DataFrame(cur.fetchall(), columns=[
    'summonerId', 'championId', 'championLevel', 'championPoints', 'lastPlayTime'
    , 'championPointsSinceLastLevel', 'championPointsUntilNextLevel', 'tokensEarned'])

In [8]:
# Drop null values
df_championMastery.dropna()

,summonerId,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,tokensEarned
0,HrertYKBrFGl4VtH1QmDWc6HeINPXkbO1OOUL7boXVoaybU,37,7,602952,1.658697e+12,581352,0,0
1,HrertYKBrFGl4VtH1QmDWc6HeINPXkbO1OOUL7boXVoaybU,99,7,87340,1.657737e+12,65740,0,0
2,HrertYKBrFGl4VtH1QmDWc6HeINPXkbO1OOUL7boXVoaybU,40,7,85539,1.654451e+12,63939,0,0
3,HrertYKBrFGl4VtH1QmDWc6HeINPXkbO1OOUL7boXVoaybU,267,5,82364,1.659732e+12,60764,0,2
4,HrertYKBrFGl4VtH1QmDWc6HeINPXkbO1OOUL7boXVoaybU,16,7,77550,1.661647e+12,55950,0,0
...,...,...,...,...,...,...,...,...
531567,zYfi7jIcNjvMvTWgF8GLs951w5M6k9zEv1agNSne_OOY-oc,497,1,152,1.613491e+12,152,1648,0
531568,zYfi7jIcNjvMvTWgF8GLs951w5M6k9zEv1agNSne_OOY-oc,89,1,150,1.601873e+12,150,1650,0
531569,zYfi7jIcNjvMvTWgF8GLs951w5M6k9zEv1agNSne_OOY-oc,32,1,145,1.549192e+12,145,1655,0
531570,zYfi7jIcNjvMvTWgF8GLs951w5M6k9zEv1agNSne_OOY-oc,161,1,134,1.602209e+12,134,1666,0


In [9]:
# Get count of unique summoners
summoner_count = len(df_championMastery["summonerId"].unique())
summoner_count

4418

In [10]:
# avg and sum of champion points by Champion
avgChampionPoints = df_championMastery.groupby(["championId"]).mean()["championPoints"]
totalChampionPoints = df_championMastery.groupby(["championId"]).sum()["championPoints"]
# count of summons by Champion
countOfSummons = df_championMastery.groupby(["championId"]).count()["summonerId"]

In [11]:
champions_summary_df = pd.DataFrame({
    "Avg Champion Points per Summon": avgChampionPoints.map("{:,.0f}".format),
    "Total Number of Points": totalChampionPoints.map("{:,.0f}".format),
    "Total Number of Summons": countOfSummons.map("{:,.0f}".format)
})

champions_summary_df.head()

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
1,"19,468","70,006,450","3,596"
10,"16,753","62,319,329","3,720"
101,"19,605","64,990,052","3,315"
102,"13,595","43,163,917","3,175"
103,"39,749","146,196,403","3,678"


In [12]:
print(champions_summary_df.dtypes)

Avg Champion Points per Summon    object
Total Number of Points            object
Total Number of Summons           object
dtype: object


In [13]:
sorted_champions_summary_df_summons = champions_summary_df.sort_values(by='Total Number of Summons', ascending=False)
sorted_champions_summary_df_summons

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
81,"71,418","291,242,515","4,078"
22,"33,043","134,286,298","4,064"
236,"65,317","261,988,482","4,011"
104,"47,561","188,152,234","3,956"
67,"67,067","263,102,216","3,923"
...,...,...,...
83,"9,154","21,163,063","2,312"
888,"8,250","15,558,576","1,886"
526,"8,568","14,874,106","1,736"


In [14]:
sorted_champions_summary_df_championPoints = champions_summary_df.sort_values(by='Total Number of Points', ascending=False)
sorted_champions_summary_df_championPoints

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
79,"28,452","98,982,991","3,479"
61,"29,097","97,097,813","3,337"
76,"26,611","96,038,791","3,609"
16,"25,659","94,911,555","3,699"
432,"30,080","94,720,442","3,149"
...,...,...,...
105,"31,803","111,500,406","3,506"
267,"36,229","107,129,369","2,957"
498,"29,824","106,352,249","3,566"


In [15]:
highestAvgChampionPoints = sorted_champions_summary_df_championPoints.loc[sorted_champions_summary_df_championPoints['Avg Champion Points per Summon'] > '65000']
highestAvgChampionPoints

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
157,"94,557","349,481,115","3,696"
81,"71,418","291,242,515","4,078"
64,"71,916","280,042,757","3,894"
887,"9,574","27,210,635","2,842"
67,"67,067","263,102,216","3,923"
202,"67,058","259,648,006","3,872"
166,"9,776","25,094,107","2,567"
412,"66,116","240,727,871","3,641"
83,"9,154","21,163,063","2,312"


In [16]:
sorted_highestAvgChampionPoints = highestAvgChampionPoints.sort_values(by='Avg Champion Points per Summon', ascending=False)
sorted_highestAvgChampionPoints

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
157,"94,557","349,481,115","3,696"
166,"9,776","25,094,107","2,567"
887,"9,574","27,210,635","2,842"
83,"9,154","21,163,063","2,312"
526,"8,568","14,874,106","1,736"
888,"8,250","15,558,576","1,886"
64,"71,916","280,042,757","3,894"
81,"71,418","291,242,515","4,078"
711,"7,416","18,858,040","2,543"


In [17]:
sorted_champions_summary_df_avgpersummons = champions_summary_df.sort_values(by='Avg Champion Points per Summon', ascending=False)
sorted_champions_summary_df_avgpersummons.head()

,Avg Champion Points per Summon,Total Number of Points,Total Number of Summons
championId,,,
157,"94,557","349,481,115","3,696"
166,"9,776","25,094,107","2,567"
887,"9,574","27,210,635","2,842"
83,"9,154","21,163,063","2,312"
526,"8,568","14,874,106","1,736"


In [18]:
# champion157 = champions_difficulty_df[(champions_difficulty_df['Champion Id'] == 157)]
# champion157

In [19]:
# champion64 = champions_difficulty_df[(champions_difficulty_df['Champion Id'] == 64)]
# champion64

In [20]:
# champion81 = champions_difficulty_df[(champions_difficulty_df['Champion Id'] == 81)]
# champion81

In [21]:
# Select champions table
query_champions = sql.SQL("""
SELECT * FROM champions;
""")

In [22]:
# connect to table
cur = conn.cursor()
cur.execute(query_champions)

In [23]:
# Test print one row
print(cur.fetchone())

('Annie', 1, 2, 3, 10, 6, 'Mage', 'n/a', 524.0, 88.0, 418.0, 25.0, 335, 19.22, 4.0, 30.0, 0.5, 625, 5.5, 0.55, 8.0, 0.8, 0, 0, 50.41, 2.625, 1.36, 0.579)


In [24]:
# Test print all rows
cur = conn.cursor()
cur.execute(query_champions)
print(cur.fetchall())

[('Annie', 1, 2, 3, 10, 6, 'Mage', 'n/a', 524.0, 88.0, 418.0, 25.0, 335, 19.22, 4.0, 30.0, 0.5, 625, 5.5, 0.55, 8.0, 0.8, 0, 0, 50.41, 2.625, 1.36, 0.579), ('Olaf', 2, 9, 5, 3, 3, 'Fighter', 'Tank', 597.24, 93.0, 315.6, 42.0, 350, 35.0, 3.0, 32.1, 1.25, 125, 8.5, 0.9, 7.466, 0.575, 0, 0, 68.0, 3.5, 2.7, 0.694), ('Galio', 3, 1, 10, 6, 5, 'Tank', 'Mage', 562.0, 112.0, 500.0, 40.0, 335, 24.0, 3.5, 32.0, 1.25, 150, 8.0, 0.8, 9.5, 0.7, 0, 0, 59.0, 3.5, 1.5, 0.625), ('TwistedFate', 4, 6, 2, 6, 9, 'Mage', 'n/a', 534.0, 94.0, 333.0, 19.0, 335, 21.0, 3.15, 30.0, 0.5, 525, 5.5, 0.6, 8.0, 0.8, 0, 0, 52.0, 3.3, 3.22, 0.651), ('XinZhao', 5, 8, 6, 3, 2, 'Fighter', 'Assassin', 570.0, 92.0, 273.8, 35.0, 345, 35.0, 3.5, 32.1, 1.25, 175, 8.0, 0.7, 7.256, 0.45, 0, 0, 66.0, 3.0, 3.5, 0.645), ('Urgot', 6, 8, 5, 3, 8, 'Fighter', 'Tank', 585.0, 88.0, 340.0, 45.0, 330, 36.0, 4.25, 32.1, 1.25, 350, 7.5, 0.7, 7.25, 0.8, 0, 0, 63.0, 4.0, 3.75, 0.625), ('Leblanc', 7, 1, 4, 10, 9, 'Assassin', 'Mage', 528.0, 92.0, 

In [25]:
# Fetch two rows
cur = conn.cursor()
cur.execute(query_champions)
rows_to_fetch = 2
print(cur.fetchmany(rows_to_fetch))

[('Annie', 1, 2, 3, 10, 6, 'Mage', 'n/a', 524.0, 88.0, 418.0, 25.0, 335, 19.22, 4.0, 30.0, 0.5, 625, 5.5, 0.55, 8.0, 0.8, 0, 0, 50.41, 2.625, 1.36, 0.579), ('Olaf', 2, 9, 5, 3, 3, 'Fighter', 'Tank', 597.24, 93.0, 315.6, 42.0, 350, 35.0, 3.0, 32.1, 1.25, 125, 8.5, 0.9, 7.466, 0.575, 0, 0, 68.0, 3.5, 2.7, 0.694)]


In [26]:
#Build champions dataframe
df_champions = pd.DataFrame(cur.fetchall(), columns=[
    'id', 'key', 'attack', 'defense', 'magic'
	, 'difficulty', 'tag1', 'tag2', 'hp', 'hpperlevel'
  , 'mp', 'mpperlevel', 'movespeed', 'armor', 'armorperlevel'
  , 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel'
  , 'mpregen', 'mpregenperlevel', 'crit', 'critperlevel'
  , 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed'])

In [27]:
#check dataFrame
df_champions

,id,key,attack,defense,magic,difficulty,tag1,tag2,hp,hpperlevel,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,Galio,3,1,10,6,5,Tank,Mage,562.0,112.0,...,8.00,0.80,9.500,0.70,0,0,59.00,3.5,1.50,0.625
1,TwistedFate,4,6,2,6,9,Mage,n/a,534.0,94.0,...,5.50,0.60,8.000,0.80,0,0,52.00,3.3,3.22,0.651
2,XinZhao,5,8,6,3,2,Fighter,Assassin,570.0,92.0,...,8.00,0.70,7.256,0.45,0,0,66.00,3.0,3.50,0.645
3,Urgot,6,8,5,3,8,Fighter,Tank,585.0,88.0,...,7.50,0.70,7.250,0.80,0,0,63.00,4.0,3.75,0.625
4,Leblanc,7,1,4,10,9,Assassin,Mage,528.0,92.0,...,7.50,0.55,6.000,0.80,0,0,54.88,3.5,1.40,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Aphelios,523,6,2,1,10,Marksman,Marksman,530.0,88.0,...,3.25,0.55,6.500,0.40,0,0,57.00,2.0,2.10,0.640
144,Pyke,555,9,3,1,7,Support,Assassin,600.0,90.0,...,7.00,0.50,8.000,1.00,0,0,62.00,2.0,2.50,0.667
145,Yone,777,8,4,4,8,Assassin,Fighter,550.0,85.0,...,7.50,0.75,0.000,0.00,0,0,60.00,3.0,2.50,0.625
146,Sett,875,8,5,1,2,Fighter,Tank,600.0,93.0,...,7.00,0.50,0.000,0.00,0,0,60.00,4.0,1.75,0.625


In [28]:
# Drop null values
df_champions.dropna()

,id,key,attack,defense,magic,difficulty,tag1,tag2,hp,hpperlevel,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
0,Galio,3,1,10,6,5,Tank,Mage,562.0,112.0,...,8.00,0.80,9.500,0.70,0,0,59.00,3.5,1.50,0.625
1,TwistedFate,4,6,2,6,9,Mage,n/a,534.0,94.0,...,5.50,0.60,8.000,0.80,0,0,52.00,3.3,3.22,0.651
2,XinZhao,5,8,6,3,2,Fighter,Assassin,570.0,92.0,...,8.00,0.70,7.256,0.45,0,0,66.00,3.0,3.50,0.645
3,Urgot,6,8,5,3,8,Fighter,Tank,585.0,88.0,...,7.50,0.70,7.250,0.80,0,0,63.00,4.0,3.75,0.625
4,Leblanc,7,1,4,10,9,Assassin,Mage,528.0,92.0,...,7.50,0.55,6.000,0.80,0,0,54.88,3.5,1.40,0.625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,Aphelios,523,6,2,1,10,Marksman,Marksman,530.0,88.0,...,3.25,0.55,6.500,0.40,0,0,57.00,2.0,2.10,0.640
144,Pyke,555,9,3,1,7,Support,Assassin,600.0,90.0,...,7.00,0.50,8.000,1.00,0,0,62.00,2.0,2.50,0.667
145,Yone,777,8,4,4,8,Assassin,Fighter,550.0,85.0,...,7.50,0.75,0.000,0.00,0,0,60.00,3.0,2.50,0.625
146,Sett,875,8,5,1,2,Fighter,Tank,600.0,93.0,...,7.00,0.50,0.000,0.00,0,0,60.00,4.0,1.75,0.625


In [29]:
# Print statistics
df_champions.describe()

,key,attack,defense,magic,difficulty,hp,hpperlevel,mp,mpperlevel,movespeed,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,...,148.000000,148.000000,148.000000,148.000000,148.0,148.0,148.000000,148.000000,148.000000,148.000000
mean,140.216216,5.466216,4.682432,5.378378,5.790541,558.009730,88.993243,310.172027,32.864865,336.270270,...,6.614865,0.665203,8.332358,0.549662,0.0,0.0,59.470094,3.225315,2.473291,0.647081
std,163.968121,2.788254,2.093193,2.738999,2.097142,37.327847,7.074431,119.220728,17.041725,7.507452,...,1.798615,0.160427,8.364865,0.289190,0.0,0.0,5.906604,0.645631,0.828711,0.034326
min,3.000000,0.000000,1.000000,1.000000,1.000000,340.000000,65.000000,0.000000,0.000000,315.000000,...,2.500000,0.400000,0.000000,0.000000,0.0,0.0,40.368000,0.000000,0.000000,0.475000
25%,39.750000,3.000000,3.000000,3.000000,4.000000,534.000000,85.000000,280.000000,25.000000,330.000000,...,5.500000,0.550000,6.500000,0.400000,0.0,0.0,55.000000,3.000000,2.000000,0.625000
50%,85.500000,6.000000,5.000000,6.000000,6.000000,565.000000,90.000000,325.300000,40.000000,335.000000,...,7.000000,0.600000,7.583000,0.600000,0.0,0.0,60.000000,3.110000,2.500000,0.639000
75%,151.000000,8.000000,6.000000,8.000000,7.000000,582.950000,93.000000,375.000000,45.000000,340.000000,...,8.000000,0.750000,8.002500,0.800000,0.0,0.0,64.000000,3.500000,3.025000,0.658000
max,876.000000,10.000000,10.000000,10.000000,10.000000,625.640000,115.000000,500.000000,67.000000,355.000000,...,10.000000,1.750000,50.000000,1.000000,0.0,0.0,70.000000,5.000000,6.000000,0.800000


In [30]:
# Calculate avg score for attack, defense, magic and difficulty scores
df_champion_element = df_champions[['id', 'attack', 'defense', 'magic', 'difficulty']]
df_champion_element['elementAvgScore'] = df_champion_element.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
# Sort by average element score
sorted_df_champion_element = df_champion_element.sort_values(by='elementAvgScore', ascending=False)
sorted_df_champion_element

,id,attack,defense,magic,difficulty,elementAvgScore
59,Rumble,3,6,8,10,6.75
32,Shaco,8,4,6,9,6.75
9,Alistar,6,9,5,7,6.75
53,Elise,6,5,7,9,6.75
7,Kayle,6,6,7,7,6.50
...,...,...,...,...,...,...
118,Kindred,8,2,2,4,4.00
19,Ashe,7,3,2,4,4.00
22,Morgana,1,6,8,1,4.00
131,Yuumi,5,1,8,2,4.00


In [32]:
df_champion_stats = df_champions[['id', 'hp', 'mp', 'movespeed', 'armor', 'spellblock', 'attackrange', 'hpregen', 'mpregen', 'crit', 'attackdamage', 'attackspeed']]
df_champion_stats['statsAvgScore'] = df_champion_stats.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [33]:
sorted_df_champion_stats = df_champion_stats.sort_values(by='statsAvgScore', ascending=False)
sorted_df_champion_stats

,id,hp,mp,movespeed,armor,spellblock,attackrange,hpregen,mpregen,crit,attackdamage,attackspeed,statsAvgScore
6,Fiddlesticks,580.40,500.0,335,34.000,30.0,480,5.5,8.0,0,55.360,0.625,184.444091
101,Lissandra,550.00,475.0,325,22.000,30.0,550,7.0,8.0,0,53.000,0.656,183.696000
31,Anivia,480.00,495.0,325,21.220,30.0,600,5.5,8.0,0,51.376,0.625,183.338273
94,Ziggs,536.00,480.0,325,21.544,30.0,550,6.5,8.0,0,54.208,0.656,182.900727
130,Azir,552.00,480.0,335,19.040,30.0,525,7.0,8.0,0,52.000,0.625,182.605909
...,...,...,...,...,...,...,...,...,...,...,...,...,...
146,Sett,600.00,0.0,340,33.000,32.0,125,7.0,0.0,0,60.000,0.625,108.875000
48,Katarina,602.00,0.0,340,27.880,32.1,125,7.5,0.0,0,58.000,0.658,108.467091
33,DrMundo,582.52,0.0,345,36.000,32.1,125,8.0,0.0,0,61.270,0.721,108.237364
78,Riven,560.00,0.0,340,33.000,32.1,125,8.5,0.0,0,64.000,0.625,105.747727


In [34]:
df_champion_statsperLevel = df_champions[['id', 'hpperlevel', 'mpperlevel', 'armorperlevel','spellblockperlevel', 'hpregenperlevel', 'mpregenperlevel', 'critperlevel', 'attackdamageperlevel', 'attackspeedperlevel']]
df_champion_statsperLevel['statsperLevelAvgScore'] = df_champion_statsperLevel.mean(axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [35]:
sorted_df_champion_statsperLevel = df_champion_statsperLevel.sort_values(by='statsperLevelAvgScore', ascending=False)
sorted_df_champion_statsperLevel

,id,hpperlevel,mpperlevel,armorperlevel,spellblockperlevel,hpregenperlevel,mpregenperlevel,critperlevel,attackdamageperlevel,attackspeedperlevel,statsperLevelAvgScore
141,Sylas,115.0,50.0,4.00,1.75,0.90,0.80,0,3.0,3.50,19.883333
35,Kassadin,90.0,67.0,2.80,0.50,0.50,0.80,0,3.9,3.70,18.800000
135,Ivern,95.0,60.0,3.50,1.25,0.85,0.75,0,3.0,3.40,18.638889
0,Galio,112.0,40.0,3.50,1.25,0.80,0.70,0,3.5,1.50,18.138889
22,Morgana,90.0,60.0,3.80,0.50,0.40,0.40,0,3.5,1.53,17.792222
...,...,...,...,...,...,...,...,...,...,...,...
145,Yone,85.0,0.0,3.40,1.25,0.75,0.00,0,3.0,2.50,10.655556
134,RekSai,85.0,0.0,3.75,1.25,0.65,0.00,0,3.0,2.00,10.627778
59,Rumble,85.0,0.0,3.50,1.25,0.60,0.00,0,3.2,1.85,10.600000
124,Kled,70.0,0.0,4.00,1.25,0.75,0.00,0,3.5,3.50,9.222222


In [36]:
# Define new column names
championName = sorted_df_champion_element['id']
difficulty = sorted_df_champion_element['difficulty']
elementAvg = sorted_df_champion_element['elementAvgScore']
statsAvg = sorted_df_champion_stats['statsAvgScore']
statsPerLevelAvg = sorted_df_champion_statsperLevel['statsperLevelAvgScore']

In [37]:
# Create new data frame
new_champions_df = pd.DataFrame({
    "Champion Name": championName,
    "Difficulty": difficulty,
    "Avg Element Score": elementAvg,
    "Avg Stats Score": statsAvg,
    "Avg Stats per Level Score": statsPerLevelAvg
})

In [38]:
# Check new data frame
new_champions_df.head()

,Champion Name,Difficulty,Avg Element Score,Avg Stats Score,Avg Stats per Level Score
0,Galio,5,5.50,152.738636,18.138889
1,TwistedFate,9,5.75,167.650091,13.841111
2,XinZhao,2,4.75,137.527364,15.488889
3,Urgot,8,6.00,159.225000,16.416667
4,Leblanc,9,6.00,167.989545,16.916667


In [39]:
 # Show new data frame statistics
 new_champions_df.describe()

,Difficulty,Avg Element Score,Avg Stats Score,Avg Stats per Level Score
count,148.000000,148.000000,148.000000,148.000000
mean,5.790541,5.329392,151.280245,14.793126
std,2.097142,0.653378,21.958174,2.102243
min,1.000000,3.500000,95.338636,8.750000
25%,4.000000,5.000000,135.853409,13.820556
50%,6.000000,5.250000,147.253091,15.235556
75%,7.000000,5.750000,171.855000,16.251111
max,10.000000,6.750000,184.444091,19.883333


In [40]:
 # Show new data frame data types
 new_champions_df.dtypes

Champion Name                 object
Difficulty                     int64
Avg Element Score            float64
Avg Stats Score              float64
Avg Stats per Level Score    float64
dtype: object

In [41]:
# Champions sorted by difficulty level
sorted_new_champions_df = new_champions_df.sort_values(by='Difficulty', ascending=False)
sorted_new_champions_df

,Champion Name,Difficulty,Avg Element Score,Avg Stats Score,Avg Stats per Level Score
59,Rumble,10,6.75,117.431273,10.600000
143,Aphelios,10,4.75,170.399091,15.394444
25,Evelynn,10,5.75,135.906818,15.244444
112,Yasuo,10,6.50,112.654273,10.916667
31,Anivia,10,6.25,183.338273,13.081111
...,...,...,...,...,...
2,XinZhao,2,4.75,137.527364,15.488889
146,Sett,2,4.00,108.875000,11.611111
89,Varus,2,4.00,175.014364,15.027778
22,Morgana,1,4.00,164.829545,17.792222


In [42]:
# Champions sorted by avg stats per level
sorted_byLevel_champions_df = new_champions_df.sort_values(by='Avg Stats per Level Score', ascending=False)
sorted_byLevel_champions_df

,Champion Name,Difficulty,Avg Element Score,Avg Stats Score,Avg Stats per Level Score
141,Sylas,5,5.00,132.422273,19.883333
35,Kassadin,8,6.00,143.553818,18.800000
135,Ivern,7,5.50,178.431273,18.638889
0,Galio,5,5.50,152.738636,18.138889
22,Morgana,1,4.00,164.829545,17.792222
...,...,...,...,...,...
145,Yone,8,6.00,154.375000,10.655556
134,RekSai,3,4.50,120.024273,10.627778
59,Rumble,10,6.75,117.431273,10.600000
124,Kled,7,4.75,95.338636,9.222222


In [43]:
# Calculate averages for avg element score
avg_score_df = new_champions_df.groupby(["Difficulty"]).mean()["Avg Element Score"]
avg_score_df

Difficulty
1     4.000000
2     4.458333
3     4.750000
4     4.956522
5     5.321429
6     5.412500
7     5.522727
8     5.625000
9     6.194444
10    6.000000
Name: Avg Element Score, dtype: float64

In [44]:
# Calculate averages for avg stats
avg_stats_df = new_champions_df.groupby(["Difficulty"]).mean()["Avg Stats Score"]
avg_stats_df

Difficulty
1     168.473318
2     143.080364
3     139.115265
4     146.326407
5     150.479334
6     158.648959
7     154.750215
8     149.605418
9     164.802040
10    148.820818
Name: Avg Stats Score, dtype: float64

In [45]:
# Calculate averages for avg stats per level
avg_statsperLevel_df = new_champions_df.groupby(["Difficulty"]).mean()["Avg Stats per Level Score"]
avg_statsperLevel_df

Difficulty
1     16.548889
2     14.741667
3     15.054583
4     14.297729
5     15.186210
6     15.097667
7     15.056449
8     14.102842
9     15.232901
10    13.461667
Name: Avg Stats per Level Score, dtype: float64

In [46]:
# Sum all averages
Total_Score =  avg_score_df + avg_stats_df + avg_statsperLevel_df

In [47]:
# Assemble into DataFrame. 
difficulty_summary_df = pd.DataFrame({
          "Average Element Score": avg_score_df.map("{:.1f}".format),
          "Average Stats Score": avg_stats_df.map("{:.1f}".format),
          "Average Stats per Level Score": avg_statsperLevel_df.map("{:.1f}".format),
          "Total Score": Total_Score.map("{:.1f}".format)})

difficulty_summary_df

,Average Element Score,Average Stats Score,Average Stats per Level Score,Total Score
Difficulty,,,,
1,4.0,168.5,16.5,189.0
2,4.5,143.1,14.7,162.3
3,4.8,139.1,15.1,158.9
4,5.0,146.3,14.3,165.6
5,5.3,150.5,15.2,171.0
6,5.4,158.6,15.1,179.2
7,5.5,154.8,15.1,175.3
8,5.6,149.6,14.1,169.3
9,6.2,164.8,15.2,186.2


In [48]:
# Create new column names for new data frame
championsName = new_champions_df['Champion Name']
championsId = df_champions['key']
Role1 = df_champions['tag1']
Role2 = df_champions['tag2']
Difficulty = df_champions['difficulty']
AvgElementScore = new_champions_df['Avg Element Score']
AvgStatsScore = new_champions_df['Avg Stats Score']
AvgStatsperLevelScore	 = new_champions_df['Avg Stats per Level Score']
TotalScore = AvgElementScore + AvgStatsScore + AvgStatsperLevelScore
# NumberofSummons = champions_summary_df['Total Number of Summons']
# NumberofPoints = champions_summary_df['Total Number of Points']
# AvgChampionPoints = champions_summary_df['Avg Champion Points per Summon']

In [49]:
champions_difficulty_df = pd.DataFrame({
    'Champion Id': championsId,    
    "Champion Name": championsName,
    "Role 1": Role1,
    "Role 2": Role2,
    "Difficulty": Difficulty,
    "Average Element Score": AvgElementScore.map("{:.1f}".format),
    "Average Stats Score": AvgStatsScore.map("{:.1f}".format),
    "Average StatsperLevel Score": AvgStatsperLevelScore.map("{:.1f}".format),
    "Total Score": TotalScore.map("{:.1f}".format),
    # "Total Number of Summons": NumberofSummons,
    # "Total Number of Points": NumberofPoints,
    # "Average Points per Summon": AvgChampionPoints,
    })

champions_difficulty_df

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score
0,3,Galio,Tank,Mage,5,5.5,152.7,18.1,176.4
1,4,TwistedFate,Mage,n/a,9,5.8,167.7,13.8,187.2
2,5,XinZhao,Fighter,Assassin,2,4.8,137.5,15.5,157.8
3,6,Urgot,Fighter,Tank,8,6.0,159.2,16.4,181.6
4,7,Leblanc,Assassin,Mage,9,6.0,168.0,16.9,190.9
...,...,...,...,...,...,...,...,...,...
143,523,Aphelios,Marksman,Marksman,10,4.8,170.4,15.4,190.5
144,555,Pyke,Support,Assassin,7,5.0,150.0,16.8,171.7
145,777,Yone,Assassin,Fighter,8,6.0,154.4,10.7,171.0
146,875,Sett,Fighter,Tank,2,4.0,108.9,11.6,124.5


In [50]:
TotalScore.describe()

count    148.000000
mean     171.402763
std       22.644649
min      109.310859
25%      156.589503
50%      168.818616
75%      192.271419
max      204.653535
dtype: float64

In [51]:
# Establish the skill bins and group names.
skill_bins = [0, 165, 195, 205]
group_names = ["difficult", "intermediate", "easy"]

In [52]:
# Add Mastery Rank to dataframe
champions_difficulty_df["Mastery Rank"] = pd.cut(TotalScore, skill_bins, labels=group_names)
champions_difficulty_df.head()

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
0,3,Galio,Tank,Mage,5,5.5,152.7,18.1,176.4,intermediate
1,4,TwistedFate,Mage,n/a,9,5.8,167.7,13.8,187.2,intermediate
2,5,XinZhao,Fighter,Assassin,2,4.8,137.5,15.5,157.8,difficult
3,6,Urgot,Fighter,Tank,8,6.0,159.2,16.4,181.6,intermediate
4,7,Leblanc,Assassin,Mage,9,6.0,168.0,16.9,190.9,intermediate


In [53]:
# Create list of champions with Mastery rank 'Easy'
easyMasteryChampions = champions_difficulty_df[champions_difficulty_df['Mastery Rank'] == 'easy']
easyMasteryChampions.sort_values(by='Total Score', ascending=False)

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
6,9,Fiddlesticks,Mage,Support,9,5.8,184.4,14.5,204.7,easy
104,134,Syndra,Mage,Support,8,5.5,182.4,15.6,203.5,easy
101,127,Lissandra,Mage,n/a,6,5.2,183.7,14.4,203.3,easy
130,268,Azir,Mage,Marksman,9,6.5,182.6,13.8,202.9,easy
31,34,Anivia,Mage,Support,10,6.2,183.3,13.1,202.7,easy
135,427,Ivern,Support,Mage,7,5.5,178.4,18.6,202.6,easy
42,45,Veigar,Mage,n/a,7,5.2,182.5,14.5,202.2,easy
94,115,Ziggs,Mage,n/a,4,4.8,182.9,14.0,201.6,easy
107,142,Zoe,Mage,Support,5,5.2,181.7,14.2,201.2,easy
96,119,Draven,Marksman,Marksman,8,5.2,179.7,15.6,200.6,easy


In [54]:
intermMasteryChampions = champions_difficulty_df[champions_difficulty_df['Mastery Rank'] == 'intermediate']
intermMasteryChampions.sort_values(by='Total Score', ascending=False)

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
40,43,Karma,Mage,Support,5,5.2,172.3,17.3,194.9,intermediate
60,69,Cassiopeia,Mage,n/a,10,6.0,173.2,15.5,194.7,intermediate
121,235,Senna,Marksman,Mage,7,5.5,174.9,14.3,194.7,intermediate
117,202,Jhin,Marksman,Mage,6,6.0,171.7,17.0,194.7,intermediate
91,112,Viktor,Mage,n/a,9,6.2,174.3,14.0,194.6,intermediate
139,498,Xayah,Marksman,Marksman,5,5.5,173.4,15.5,194.3,intermediate
89,110,Varus,Marksman,Mage,2,4.0,175.0,15.0,194.0,intermediate
26,29,Twitch,Marksman,Assassin,6,5.0,173.4,15.2,193.6,intermediate
27,30,Karthus,Mage,n/a,7,5.2,174.0,14.2,193.5,intermediate
53,60,Elise,Mage,Fighter,9,6.8,169.3,17.0,193.0,intermediate


In [55]:
difficultMasteryChampions = champions_difficulty_df[champions_difficulty_df['Mastery Rank'] == 'difficult']
difficultMasteryChampions.sort_values(by='Total Score', ascending=False)

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
43,48,Trundle,Fighter,Tank,5,5.0,143.0,16.9,164.9,difficult
7,10,Kayle,Fighter,Support,7,6.5,142.1,16.0,164.6,difficult
77,91,Talon,Assassin,n/a,7,5.0,143.5,16.0,164.6,difficult
102,131,Diana,Fighter,Mage,4,6.2,143.3,14.6,164.2,difficult
87,106,Volibear,Fighter,Tank,3,5.2,141.7,16.8,163.8,difficult
...,...,...,...,...,...,...,...,...,...,...
88,107,Rengar,Assassin,Fighter,8,5.2,109.2,11.2,125.6,difficult
33,36,DrMundo,Fighter,Tank,5,5.8,108.2,11.2,125.2,difficult
146,875,Sett,Fighter,Tank,2,4.0,108.9,11.6,124.5,difficult
78,92,Riven,Fighter,Assassin,8,5.5,105.7,10.8,122.1,difficult


In [56]:
# List most difficult champions
mostdifficultChampions = champions_difficulty_df[(champions_difficulty_df['Difficulty'] == 10) 
                                             & (champions_difficulty_df['Mastery Rank'] == 'difficult')]

mostdifficultChampions

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
25,28,Evelynn,Assassin,Mage,10,5.8,135.9,15.2,156.9,difficult
59,68,Rumble,Fighter,Mage,10,6.8,117.4,10.6,134.8,difficult
112,157,Yasuo,Fighter,Assassin,10,6.5,112.7,10.9,130.1,difficult


In [57]:
# List difficult champions

difficultChampions = champions_difficulty_df[(champions_difficulty_df['Difficulty'] == 9) 
                                             & (champions_difficulty_df['Mastery Rank'] == 'difficult')]

difficultChampions

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
32,35,Shaco,Assassin,n/a,9,6.8,136.4,15.6,158.8,difficult
38,41,Gangplank,Fighter,n/a,9,6.5,130.7,14.9,152.0,difficult


In [58]:
# List most easiest champions

mostEasiestChampions = champions_difficulty_df[(champions_difficulty_df['Difficulty'] == 3) 
                                             & (champions_difficulty_df['Mastery Rank'] == 'easy')]

mostEasiestChampions

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
13,16,Soraka,Support,Mage,3,4.2,178.3,13.8,196.4,easy


In [59]:
EasyChampions = champions_difficulty_df[(champions_difficulty_df['Difficulty'] == 4) 
                                             & (champions_difficulty_df['Mastery Rank'] == 'easy')]

EasyChampions

,Champion Id,Champion Name,Role 1,Role 2,Difficulty,Average Element Score,Average Stats Score,Average StatsperLevel Score,Total Score,Mastery Rank
56,63,Brand,Mage,n/a,4,4.2,182.2,13.2,199.6,easy
94,115,Ziggs,Mage,n/a,4,4.8,182.9,14.0,201.6,easy


In [60]:
cur.close() 
conn.commit()